In [ ]:
# SPDX-License-Identifier: Apache-2.0

# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

# (C) Copyright XXX 2023.

# We honor the previous contributions and publish own contributions 
# under the same Apache 2.0 License, to the best of our knowldedge, in compliance with the Apache-2.0 license terms.


# General Notes

This Notebook is intened to wrap low level data about the mimiced (or completely fictional) hardware into config dictionaries,
which are used in the run_simulation script.


We provide a template here, which can be used to add your data, with a custom name or you can use any of the predefined backends
and play around with the parameters.
You could, for example, increase the coupling density of ibmq_kolkata / rigetti_aspenm3 backends and investigate on the influence
in the simulation results.

In case you add your own backend here; add backend name in the match statements in method:
 - custom_transpiler in run_simulation.py

in order to make it compatible with the code basis.
--> Thats it
## General imports


In [ ]:
from helper_functions import *

# Extract data from IBMQ-snapshots 

In [ ]:
import numpy as np
from qiskit.providers.fake_provider import FakeKolkataV2, FakeManilaV2, FakeWashingtonV2
from collections import defaultdict


# Choose backend
backend = FakeKolkataV2()
target = backend.target

# Collect qubit data
t1_avg = 0
t2_avg = 0
num_samps = 0
for qubit in target.qubit_properties:
    num_samps += 1
    t1_avg += qubit.t1
    t2_avg += qubit.t2
t1_avg /= num_samps
t2_avg /= num_samps

# Collect gate data
e1_avg = 0
e2_avg = 0
tg1_avg = 0
tg2_avg = 0
num_samps_q1 = 0
num_samps_q2 = 0

for op_name, inst_prop_dic in target.items():
    # NOTE:  Skip rz and z entries, since we do not apply noise to those gates in our final circuit.
    if op_name == 'reset' or op_name == 'delay' or op_name == "id" or op_name == 'measure' or op_name == "rz" or op_name == "z":
        continue
    for qubits, inst_prop in inst_prop_dic.items():
        if len(qubits) == 1:
            num_samps_q1 += 1
            tg1_avg += inst_prop.duration
            e1_avg += inst_prop.error
        elif len(qubits) == 2:
            num_samps_q2 += 1
            tg2_avg += inst_prop.duration
            e2_avg += inst_prop.error
tg1_avg /= num_samps_q1
e1_avg /= num_samps_q1
tg2_avg /= num_samps_q2
e2_avg /= num_samps_q2

# +++ Construct hardware config +++
hw_config = defaultdict()
hw_config["hw_name"] = "ibmq_" + backend.name.split("_")[1]


hw_config["qubit_count"] = target.num_qubits
hw_config["basis_gates"] = [(operations.name, operations.num_qubits)
                            for operations in target.operations if operations.name not in ["delay", 'measure', 'reset', 'id']]



hw_config["coupling_map"] = list(backend.coupling_map.get_edges())

# NOTE: Example for setting the full coupling map
# hw_config["coupling_map"] = get_full_map(target.num_qubits)


density = len(hw_config["coupling_map"])/(hw_config["qubit_count"]*(hw_config["qubit_count"]-1))

hw_config["T1_avg"] = t1_avg
hw_config["T2_avg"] = t2_avg
hw_config["TG1_avg"] = tg1_avg
hw_config["TG2_avg"] = tg2_avg
hw_config["e1_avg"] = e1_avg
hw_config["e2_avg"] = e2_avg


# +++ Save data +++
np.save("./backend_data/"+hw_config["hw_name"]+".npy", hw_config, allow_pickle=True)


# Save Rigetti data


In [ ]:
import numpy as np
from collections import defaultdict
# Import Rigetti Aspen Topology
hw_config = defaultdict()


# Rigetti Aspen Topology
hw_config["coupling_map"] = [
    (0, 1), (1, 0), (0, 7), (7, 0), (0, 19), (19, 0), (1, 2), (2, 1), (1, 16), (16, 1), (8, 9), (9, 8), (8, 39), (39, 8), (9, 18), (18, 9), (9, 54), (54, 9), (10, 11),
    (11, 10), (10, 17), (17, 10), (10, 51), (51, 10), (11, 12), (12, 11), (11, 26), (26, 11), (48, 49), (49, 48), (48, 55), (55, 48), (49, 50), (50, 49), (49, 62), (62, 49),
    (20, 21), (21, 20), (20, 27), (27, 20), (20, 59), (59, 20), (21, 22), (22, 21), (21, 36), (36, 21), (56, 57), (57, 56), (56, 63), (63, 56), (57, 58), (58, 57), (57, 70),
    (70, 57), (30, 31), (31, 30), (30, 37), (37, 30), (30, 67), (67, 30), (31, 32), (32, 31), (31, 46), (46, 31), (64, 65), (65, 64), (64, 71), (71, 64), (65, 66), (66, 65),
    (65, 78), (78, 65), (40, 41), (41, 40), (40, 47), (47, 40), (40, 75), (75, 40), (41, 42), (42, 41), (72, 73), (73, 72), (72, 79), (79, 72), (73, 74), (74, 73), (2, 3),
    (3, 2), (2, 15), (15, 2), (18, 19), (19, 18), (18, 53), (53, 18), (12, 13), (13, 12), (12, 25), (25, 12), (50, 51), (51, 50), (50, 61), (61, 50), (22, 23), (23, 22),
    (22, 35), (35, 22), (58, 59), (59, 58), (58, 69), (69, 58), (32, 33), (33, 32), (32, 45), (45, 32), (66, 67), (67, 66), (66, 77), (77, 66), (42, 43), (43, 42), (74, 75), 
    (75, 74), (3, 4), (4, 3), (19, 28), (28, 19), (13, 14), (14, 13), (51, 52), (52, 51), (23, 24), (24, 23), (59, 60), (60, 59), (33, 34), (34, 33), (67, 68), (68, 67), (43, 44), 
    (44, 43), (75, 76), (76, 75), (4, 5), (5, 4), (28, 29), (29, 28), (28, 7), (7, 28), (14, 15), (15, 14), (52, 53), (53, 52), (52, 17), (17, 52), (24, 25), (25, 24), (60, 61),
    (61, 60), (60, 27), (27, 60), (34, 35), (35, 34), (68, 69), (69, 68), (68, 37), (37, 68), (44, 45), (45, 44), (76, 77), (77, 76), (76, 47), (47, 76), (5, 6), (6, 5), (29, 38),
    (38, 29), (15, 16), (16, 15), (53, 54), (54, 53), (25, 26), (26, 25), (61, 62), (62, 61), (35, 36), (36, 35), (69, 70), (70, 69), (45, 46), (46, 45), (77, 78), (78, 77), (6, 7),
    (7, 6), (38, 39), (39, 38), (16, 17), (17, 16), (54, 55), (55, 54), (26, 27), (27, 26), (62, 63), (63, 62), (36, 37), (37, 36), (70, 71), (71, 70), (46, 47), (47, 46), (78, 79), (79, 78)]


hw_config["hw_name"] = "rigetti_aspenm3"

# NOTE: Example with rigetti_connected
# hw_config["coupling_map"] = get_full_map(80)
# hw_config["hw_name"] = "rigetti_connected"

hw_config["qubit_count"] = 80
# 1) owmXy: xx_plus_yy with beta=0 NOTE: We used the Qiskit regular xx_plus_yy; but it gets newer compiled to anyway, as far as we are aware
# 2) CPhase is qiskit-native cp gate
# 3) cz is also qiskit native
# 4) Rx added withr estricted angle set
# 5) rz also
# --> gate Set Rigetti complete


# Data from: https://qcs.rigetti.com/qpus (AspenM3)
rigetti_f1qrb = [0.998, 0.9883, 0.9992, 0.9989, 0.9993, 0.9985, 0.9992, 0.9986, 0.9982, 0.9967, 0.9975, 0.9967, 0.9975, 0.9985, 0.9991, 0.9945, 0.9986, 0.9976, 0.9991, 1, 0.997, 0.9938, 0.9947, 0.9985, 0.9967, 1, 0.9986, 0.9878, 0.9987, 0.9974, 0.9991, 0.9997, 0.9991, 0.9996, 0.9989, 0.9996, 0.9988, 0.9996, 0.9989,
                 0.997, 0.9987, 0.9992, 0.9988, 0.9995, 0.999, 0.9989, 0.9992, 0.9995, 0.9747, 0.9972, 0.9988, 0.991, 0.9971, 0.9991, 0.9973, 0.993, 0.9986, 0.9993, 0.9958, 0.9994, 0.9974, 0.9987, 0.9028, 0.9996, 0.9874, 0.9992, 0.9967, 0.9989, 0.998, 0.9994, 0.9995, 0.9987, 0.9994, 0.9991, 0.9991, 0.9985, 0.9819, 0.9992, 0.9996]
# For 2q: only cz and xy available.(No cp)
rigetti_f2qrbCz = [0.781, 0.9755, 0.8556, 0.9866, 0.882, 0.8348, 0.9661, 0.9285, 0.9708, 0.9016, 0.9353, 0.9326, 0.9204, 0.8573, 0.9611, 0.9476, 0.9786, 0.9688, 0.8271, 0.9493, 0.7251, 0.9474, 0.9717, 0.8589, 0.9751, 0.8578, 0.7862, 0.823, 0.8771, 0.8358, 0.9685, 0.9521, 0.9666, 0.8042, 0.9652, 0.9702, 0.9689, 0.973, 0.9937, 0.8075, 0.7739,
                   0.9915, 0.9451, 0.8475, 0.8244, 0.8006, 0.8405, 0.8584, 0.9828, 0.9845, 0.9843, 0.9483, 0.9893, 0.9088, 0.9744, 0.8502, 0.8719, 0.9056, 0.8189, 0.8372, 0.9212, 0.829, 0.8466, 0.9906, 0.991, 0.9753, 0.9504, 0.9745, 0.823, 0.9212, 0.775, 0.9098, 0.9386, 0.9183, 0.8183, 0.8879, 0.9024, 0.9824, 0.9824, 0.6438, 0.7433, 0.9408, 0.9153, 0.8516, 0.884]
rigetti_f2qrbXy = [0.8243, 0.7674, 0.9529, 0.8405, 0.8484, 0.8318, 0.969, 0.9699, 0.9594, 0.9571, 0.9681, 0.9491, 0.9845, 0.824, 0.9022, 0.9695, 0.851, 0.88, 0.9508, 0.8258, 0.9717, 0.8906, 0.9674, 0.9916, 0.968, 0.8643, 0.8602, 0.8254, 0.9725, 0.8025, 0.9657, 0.9711, 0.9766, 0.9676, 0.8161, 0.8476, 0.9644, 0.9719, 0.962, 0.9798, 0.9153, 0.9653, 0.9765, 0.816, 0.9862, 0.9661,
                   0.961, 0.9521, 0.9584, 0.8209, 0.7879, 0.8182, 0.9579, 0.9791, 0.9721, 0.9745, 0.9793, 0.9693, 0.986, 0.9625, 0.9785, 0.8105, 0.8317, 0.9832, 0.792, 0.8296, 0.9875, 0.7926, 0.9467, 0.7994, 0.979, 0.9863, 0.9635, 0.9705, 0.9723, 0.99, 0.946, 0.7485, 0.7103, 0.8844, 0.916, 0.8369, 0.973, 0.8883, 0.8868, 0.9332, 0.9235, 0.981, 0.9873, 0.6927, 0.7501, 0.8888, 0.8952, 0.7465, 0.9509]

f2qrbtotal = rigetti_f2qrbCz + rigetti_f2qrbXy

e1_mean = 1 - np.average(rigetti_f1qrb)
e2_mean = 1 - np.average(f2qrbtotal)

# NOTE: For Rigetti the Rx gate could only handle +-pi and +- pi/2! --> Thus we have custom Rx gate to cover that.
hw_config["basis_gates"] = [('x', 1), ('OwnX1', 1), ('OwnX3', 1),
                            ('rz', 1), ('cz', 2), ('cp', 2), ('xx_plus_yy', 2)]

hw_config["T1_avg"] = 24.98e-6
hw_config["T2_avg"] = 28.04e-6  
hw_config["TG1_avg"] = 40e-9  # These are taken from the text on the website
hw_config["TG2_avg"] = 240e-9  # These are taken from the text on the website
hw_config["e1_avg"] = e1_mean
hw_config["e2_avg"] = e2_mean


density = len(hw_config["coupling_map"])/(hw_config["qubit_count"]*(hw_config["qubit_count"]-1))
# +++ Save data +++
np.save("./backend_data/"+hw_config["hw_name"] +
        ".npy", hw_config, allow_pickle=True)


# Save IONQ data

In [ ]:
from collections import defaultdict
import numpy as np

hw_config = defaultdict()
hw_config["hw_name"] = "ionq_aria"
hw_config["qubit_count"] = 21


# 1) GPI1 (custom R Gate with theta=pi)
# 2) GPI2 (custom R Gate with theta = pi/2)
# 3) RZ Gate --> native
# 4) MS Gate : IONQ's ms gate is equivalent to Rxx for some angle, which we use in custom compiler
# https://ionq.com/posts/july-25-2022-ionq-aria-part-one-practical-performance


hw_config["basis_gates"] = [
    ("OwnGPI1", 1), ("rz", 1), ("OwnGPI2", 1), ("ms", 2)]
# NOTE: Since for Ionq we have a custom compilation process, the gates provided here do not influence anything.

hw_config["coupling_map"] = get_full_map(21) 
hw_config["T1_avg"] = 50
hw_config["T2_avg"] = 1
hw_config["TG1_avg"] = 135e-6
hw_config["TG2_avg"] = 600e-6
hw_config["e1_avg"] = 0.05e-2
hw_config["e2_avg"] = 0.4e-2

# +++ Save data +++
np.save("./backend_data/"+hw_config["hw_name"] +
        ".npy", hw_config, allow_pickle=True)


# Default for RX
This backend is for simple experiments involving a single-qubit, single-gate variational circuit (which did not make it into the paper)

In [ ]:
from collections import defaultdict
import numpy as np


hw_config = defaultdict()
hw_config["hw_name"] = "simple_simple"
hw_config["qubit_count"] = 1



hw_config["basis_gates"] = [
    ("rx",1)]

hw_config["coupling_map"] = get_full_map(1) 
hw_config["T1_avg"] = 50
hw_config["T2_avg"] = 1
hw_config["TG1_avg"] = 135e-6
hw_config["TG2_avg"] = 600e-6
hw_config["e1_avg"] = 0.05e-2
hw_config["e2_avg"] = 0.4e-2

# +++ Save data +++
np.save("./backend_data/"+hw_config["hw_name"] +
        ".npy", hw_config, allow_pickle=True)

# Custom configs | e.g. predictions about future gate accuracies

In [ ]:
from collections import defaultdict
import numpy as np
from helper_functions import *


hw_config = defaultdict()
hw_config["hw_name"] = "config_template"
hw_config["qubit_count"] = 30

# NOTE: Add the basis gates here; For custom gates, see how the procedure worked for IONQ and Rigetti
hw_config["basis_gates"] = [("id", 1), ("rz", 1), ("sx", 1), ("cx", 2)]

# NOTE: Here you can set your own coupling map
hw_config["coupling_map"] = get_line_map(30) 

# We quickly show how to increase the density here, by adding random edges
# (Uncomment to test out)
# density_wish = 0.1
# base_map = get_line_map(30)
# for i in range(len(base_map)):
#     base_map[i] = list(base_map[i])

# dense_map = increase_coupling_density(base_map,density_wish)
# for i in range(len(dense_map)):
#     dense_map[i] = tuple(dense_map[i])
# hw_config["coupling_map"] = dense_map

# NOTE: Edit these params for your own experiments
hw_config["T1_avg"] = 50
hw_config["T2_avg"] = 1
hw_config["TG1_avg"] = 135e-6
hw_config["TG2_avg"] = 600e-6
hw_config["e1_avg"] = 0.05e-2
hw_config["e2_avg"] = 0.4e-2

# +++ Save data +++
np.save("./backend_data/"+hw_config["hw_name"] +
        ".npy", hw_config, allow_pickle=True)


#